# Preprocessing Data Kesejahteraan

Notebook ini melakukan proses pembersihan dan persiapan data sebelum digunakan untuk pemodelan.

**Langkah-langkah:**
1. Memuat dataset mentah.
2. Membersihkan data (handling missing values).
3. Seleksi fitur dan preprocessing awal.
4. Menyimpan data yang sudah bersih.

In [19]:
import pandas as pd
import numpy as np
import os
import sys

# Konfigurasi Path relatif terhadap notebook ini
DATA_PATH = '../dataset/dataset_akhir/dataset_final.csv'
OUTPUT_PATH = 'dataset_preprocessed.csv'

## 1. Load Data
Fungsi untuk memuat data dari file CSV.

In [20]:
def load_data(path):
    """Memuat dataset dari path yang diberikan."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File dataset tidak ditemukan di: {path}")
    
    try:
        df = pd.read_csv(path)
        print(f"✓ Dataset dimuat: {df.shape[0]} baris, {df.shape[1]} kolom")
        return df
    except Exception as e:
        raise Exception(f"Gagal memuat dataset: {str(e)}")

## 2. Clean Data
Membersihkan data, terutama menghapus baris yang tidak memiliki target label.

In [21]:
def clean_data(df, target_col='kesejahteraan'):
    """Membersihkan data: handling missing values."""
    df_clean = df.copy()
    
    # Drop baris jika target kosong (karena kita butuh label untuk training)
    if target_col in df_clean.columns:
        initial_rows = len(df_clean)
        df_clean = df_clean.dropna(subset=[target_col])
        dropped_rows = initial_rows - len(df_clean)
        if dropped_rows > 0:
            print(f"  - Menghapus {dropped_rows} baris dengan target '{target_col}' kosong.")
    
    return df_clean

## 3. Preprocessing & Feature Selection
Memilih fitur yang relevan dan mengisi nilai yang hilang pada fitur tersebut.

In [22]:
def preprocess_data(df):
    """
    Melakukan seleksi fitur dan preprocessing lainnya.
    Fitur: jumlah_penduduk_miskin, jumlah_pengangguran_terbuka, pdrb_total_adhk, harapan_lama_sekolah
    Target: kesejahteraan
    """
    selected_features = [
        'jumlah_penduduk_miskin',
        'jumlah_pengangguran_terbuka',
        'pdrb_total_adhk',
        'harapan_lama_sekolah'
    ]
    target = 'kesejahteraan'
    
    # Validasi kolom
    missing_cols = [col for col in selected_features + [target] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Kolom berikut tidak ditemukan di dataset: {missing_cols}")
    
    df_model = df[selected_features + [target]].copy()
    
    # Isi missing values di fitur dengan median
    for col in selected_features:
        if df_model[col].isnull().sum() > 0:
            median_val = df_model[col].median()
            df_model[col] = df_model[col].fillna(median_val)
            print(f"  - Mengisi missing values di '{col}' dengan median: {median_val}")

    print("✓ Preprocessing selesai.")
    return df_model

## 4. Eksekusi Pipeline
Menjalankan seluruh proses dan menyimpan hasilnya.

In [23]:
# Load
df = load_data(DATA_PATH)

# Clean
df_clean = clean_data(df)

# Preprocess
df_final = preprocess_data(df_clean)

# Preview Data
print("\nPreview Data Hasil Preprocessing:")
display(df_final.head())

✓ Dataset dimuat: 405 baris, 17 kolom
✓ Preprocessing selesai.

Preview Data Hasil Preprocessing:


,jumlah_penduduk_miskin,jumlah_pengangguran_terbuka,pdrb_total_adhk,harapan_lama_sekolah,kesejahteraan
0,477200,686459,9.293200e+10,3,Sangat Sejahtera
1,249600,255889,2.860100e+10,3,Tidak Sejahtera
2,311000,302214,1.969700e+10,3,Sangat Tidak Sejahtera
3,296300,556729,4.843200e+10,3,Cukup
4,335600,295040,2.546500e+10,3,Sangat Tidak Sejahtera


In [24]:
# Save Data
df_final.to_csv(OUTPUT_PATH, index=False)
print(f"\n✓ Data tersimpan di: {os.path.abspath(OUTPUT_PATH)}")
print(f"  Shape Akhir: {df_final.shape}")


✓ Data tersimpan di: /media/boba/DATA/STT Wastukancana/SEMESTER 5/MACHINE LEARNINGGGGG/TUGAS/UAS/prediksi_kesejahteraan/preprocessing/dataset_preprocessed.csv
  Shape Akhir: (405, 5)
